# Electricity production in Germany 2015-2022

For a [story for BBC](https://www.bbc.com/news/science-environment-64179918), [Nassos Stylianou](https://twitter.com/nassos_/status/1611300724184580096) and [Erwan Rivault](https://twitter.com/ErwanRivault) showed some interesting heatmap charts about changes in the electricity production in Great Britain. I adapted (and extended) those charts to Germany using official data available since 2015 from [SMARD](https://www.smard.de/home/downloadcenter/download-marktdaten/), a service provided by the Federal Network Agency (Bundesnetzagentur).

In [ ]:
# Import necessary libraries
import datetime as dt

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image
from plotly.subplots import make_subplots

In [ ]:
# Define a template for all plots
custom_template = {
    "layout": go.Layout(
        font={
            "family": "Lato",
            "size": 12,
            "color": "#1f1f1f",
        },
        title={
            "font": {
                "family": "Lato",
                "size": 24,
                "color": "#1f1f1f",
            },
        },
    )
}

# Assign logo URI to variable
logo = "data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiIHN0YW5kYWxvbmU9Im5vIj8+CjwhLS0gQ3JlYXRlZCB3aXRoIElua3NjYXBlIChodHRwOi8vd3d3Lmlua3NjYXBlLm9yZy8pIC0tPgoKPHN2ZwogICB3aWR0aD0iMTUwbW0iCiAgIGhlaWdodD0iMTUwbW0iCiAgIHZpZXdCb3g9IjAgMCAxNTAgMTUwIgogICB2ZXJzaW9uPSIxLjEiCiAgIGlkPSJzdmc1IgogICB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciCiAgIHhtbG5zOnN2Zz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgogIDxkZWZzCiAgICAgaWQ9ImRlZnMyIiAvPgogIDxnCiAgICAgaWQ9ImxheWVyMSI+CiAgICA8ZwogICAgICAgaWQ9ImcxOTg5Ij4KICAgICAgPGcKICAgICAgICAgaWQ9ImcyNzY0IgogICAgICAgICB0cmFuc2Zvcm09Im1hdHJpeCgwLjk1NjQyMTM2LDAsMCwwLjk1NTE1NjA0LC0zMC4zNDQ2NDgsLTM1LjA2MjUxNykiCiAgICAgICAgIHN0eWxlPSJzdHJva2Utd2lkdGg6MS4wNDYyNSI+CiAgICAgICAgPHJlY3QKICAgICAgICAgICBzdHlsZT0iZmlsbDojZmZmZmZmO3N0cm9rZTpub25lO3N0cm9rZS13aWR0aDowLjI3NzI1NyIKICAgICAgICAgICBpZD0icmVjdDMxNzQiCiAgICAgICAgICAgd2lkdGg9IjE1Ni44MzQ2NCIKICAgICAgICAgICBoZWlnaHQ9IjE1Ny4wNDI0IgogICAgICAgICAgIHg9IjMxLjcyNzI4IgogICAgICAgICAgIHk9IjM2LjcwODY3OSIgLz4KICAgICAgICA8cGF0aAogICAgICAgICAgIGlkPSJwYXRoMTU1OCIKICAgICAgICAgICBzdHlsZT0iZmlsbDojMDAwMDAwO3N0cm9rZTpub25lO3N0cm9rZS13aWR0aDoxLjAwMTU3IgogICAgICAgICAgIGQ9Ik0gMjgzLjQ2NDg0IDAgQSAyODMuNDY0NTcgMjgzLjA4OTU1IDAgMCAwIDAgMjgzLjA4OTg0IEEgMjgzLjQ2NDU3IDI4My4wODk1NSAwIDAgMCA1NC44MDY2NDEgNDUwLjMzMzk4IEMgOTAuMjM1NzU1IDQ0NS44NjA4OSAxMDYuNTg0NTUgNDQ0LjYwMjk5IDEyMi4xMTMyOCA0NDEuNzAxMTcgQyAxMzMuMzQ1ODMgNDM5LjU2NDI4IDEzNC42NjQxMiA0MzkuMDEwNzYgMTM1Ljk5NjA5IDQzNS44ODA4NiBDIDEzNi42MTA5MSA0MzQuNDM2MTYgMTM1Ljc4MjM3IDQzMS42NjA4IDEzNi4wMjczNCA0MzAuMzk4NDQgQyAxMzYuMzA3OTYgNDI4Ljk1MjI2IDEzNy41MDkyNCA0MjcuMjcwOTMgMTM5LjI3NTM5IDQyNS44NTM1MiBDIDE0MS4wNzkzMyA0MjQuNDA1NzEgMTQyLjI0ODA2IDQyMi43NDk4MiAxNDIuNTU2NjQgNDIxLjIwODk4IEMgMTQzLjY2MTg4IDQxNS42OTAxIDE0NC42MDk4MyA0MTMuMjczMDkgMTQ2LjY5NTMxIDQxMC42NTIzNCBDIDE0Ny45MDgxMiA0MDkuMTI4MjIgMTQ4LjkwMDM5IDQwNy4wODQwMSAxNDguOTAwMzkgNDA2LjEwOTM4IEMgMTQ4LjkwMDM5IDQwMi4zNTY2MyAxNTYuNDg1NzQgMzkxLjUyNTY4IDE1OS42NzM4MyAzOTAuNzI2NTYgQyAxNjEuMjQzODQgMzkwLjMzMzA3IDE2OS44NTc5NiAzOTAuNjU0MzkgMTc2Ljg1MzUyIDM5MS4zNjcxOSBDIDE3OC42Nzc2IDM5MS41NTMxIDE3OC42OTM3NCAzOTEuNTAwOTggMTc4LjkwNjI1IDM4NS4wMDk3NyBDIDE3OS4wMjQwNiAzODEuNDEwMiAxNzguODM1NDUgMzc4LjE3ODc0IDE3OC40ODYzMyAzNzcuODMwMDggQyAxNzguMTM3MTcgMzc3LjQ4MTM1IDE3Ni4zODUzNiAzNzYuOTQ3NzQgMTc0LjU5Mzc1IDM3Ni42NDQ1MyBDIDE3MS41Mzc5NyAzNzYuMTI3MzYgMTcxLjMyMjE2IDM3NS45MzU4IDE3MS4wOTU3IDM3My41MzkwNiBDIDE3MC45NjI5MyAzNzIuMTMzODIgMTcwLjQyNzcgMzcwLjQ4ODg3IDE2OS45MDYyNSAzNjkuODgyODEgQyAxNjkuMzg0ODEgMzY5LjI3NjcyIDE2OC43MzEzNSAzNjcuMjk1NzQgMTY4LjQ1NTA4IDM2NS40ODI0MiBDIDE2OC4xNzg4IDM2My42NjkwNiAxNjYuODExODggMzU5LjkwNTYyIDE2NS40MTYwMiAzNTcuMTE5MTQgQyAxNjQuMDIwMTYgMzU0LjMzMjcgMTYyLjg3Njk1IDM1MS40Mzc0MSAxNjIuODc2OTUgMzUwLjY4MzU5IEMgMTYyLjg3Njk1IDM0OS45Mjk3OCAxNjIuNjIxMTcgMzQ5LjMxMjUgMTYyLjMwODU5IDM0OS4zMTI1IEMgMTYxLjU1MjAyIDM0OS4zMTI1IDE1OS42NzMyMSAzNDUuNTQwNjggMTU3LjczMjQyIDM0MC4xMjg5MSBDIDE1Ni44NjMwMSAzMzcuNzA0NjYgMTU1LjAxNTg4IDMzNC4yNjIwNiAxNTMuNjI4OTEgMzMyLjQ3ODUyIEMgMTUyLjI0MTkzIDMzMC42OTUwNSAxNTEuMDkzOTMgMzI4LjcxMTA5IDE1MS4wNzYxNyAzMjguMDcwMzEgQyAxNTEuMDU4NDYgMzI3LjQyOTU3IDE1MC41NDg0MiAzMjUuNTgyOTggMTQ5Ljk0MzM2IDMyMy45NjY4IEMgMTQ5LjMzODI5IDMyMi4zNTA1OCAxNDguNzAzNzUgMzIwLjQxOTQ4IDE0OC41MzMyIDMxOS42NzM4MyBDIDE0OC4zNjI2NiAzMTguOTI4MjEgMTQ2LjkzNTcxIDMxNS42MjI5NCAxNDUuMzYzMjggMzEyLjMyODEyIEMgMTQzLjc5MDg1IDMwOS4wMzMyOCAxNDEuOTEyMSAzMDQuOTQxNTMgMTQxLjE4NzUgMzAzLjIzNjMzIEMgMTM5LjYyNDM4IDI5OS41NTc3NyAxMzguODI2NjQgMjk4LjgxMTI4IDEzMy4yNTc4MSAyOTUuODA2NjQgQyAxMjUuODY4MTYgMjkxLjgxOTYxIDEyMS45OTY3IDI4Ny45NTg5NCAxMTMuMjQwMjMgMjc1Ljg0OTYxIEMgMTEyLjIxNzYzIDI3NC40MzU0OSAxMDkuOTYyNCAyNzEuNDYwNTEgMTA4LjIyODUyIDI2OS4yMzgyOCBDIDEwNi40OTQ2MyAyNjcuMDE2MDEgMTA0LjY0NzYxIDI2NC4xNTYwNCAxMDQuMTI1IDI2Mi44ODI4MSBDIDEwMy42MDIzOCAyNjEuNjA5NTUgMTAyLjM3MzQ2IDI1OS44MTY4NyAxMDEuMzk0NTMgMjU4Ljg5ODQ0IEMgMTAwLjQxNTYxIDI1Ny45Nzk5NyA5OS42MDg5MjIgMjU2LjcwNjk0IDk5LjYwMTU2MiAyNTYuMDcwMzEgQyA5OS41OTQzMzMgMjU1LjQzMzY1IDk4LjYwMDkxMyAyNTMuNDU3NjcgOTcuMzk0NTMxIDI1MS42Nzk2OSBDIDk1LjU2ODI5NSAyNDguOTg4MTEgOTUuMjAxMTcyIDI0Ny43MTQ3MiA5NS4yMDExNzIgMjQ0LjA3MjI3IEMgOTUuMjAxMTcyIDI0MC40MzI3IDk1LjUyNDEwNiAyMzkuMzE2MTggOTcuMTE1MjM0IDIzNy40Mjc3MyBDIDk5LjMzMjQ2MSAyMzQuNzk2MjMgMTAxLjg3MzcxIDIzMy4yODk1NSAxMDUuNSAyMzIuNDU4OTggQyAxMDYuOTE2MDQgMjMyLjEzNDggMTA5LjM5OTI1IDIzMS40OTc2OSAxMTEuMDE3NTggMjMxLjA0Mjk3IEMgMTEyLjYzNTkgMjMwLjU4ODI4IDExNy42MDAzOSAyMjkuNzU2MzQgMTIyLjA1MDc4IDIyOS4xOTMzNiBDIDEzMS4wMDIyOSAyMjguMDYwOTkgMTM1LjE0MTc5IDIyNy4zNzExNiAxMzUuNTgzOTggMjI2Ljk0MTQxIEMgMTM2LjA1NDQ0IDIyNi40ODQxNyAxMzUuNjU3ODQgMjA2LjQ3MTM0IDEzNC42MjMwNSAxNzguNTExNzIgQyAxMzQuMTc1NDQgMTY2LjQxNzIgMTM1LjU5OTc1IDE1Ni43NzcxMSAxMzguNzQ4MDUgMTUwLjU5NTcgQyAxMzkuNzc2OTggMTQ4LjU3NTQ2IDE0MC44NzM5NSAxNDYuMDk2MDQgMTQxLjE4NTU1IDE0NS4wODU5NCBDIDE0Mi43MjA2OCAxNDAuMTA5NTIgMTYzLjQ3MDgzIDExNi45MjU4NyAxNjguODM3ODkgMTE0LjE5MTQxIEMgMTcwLjM2NjkxIDExMy40MTIzOSAxNzIuODMzNTIgMTExLjk1MTYyIDE3NC4zMTgzNiAxMTAuOTQ1MzEgQyAxNzcuODEwNDQgMTA4LjU3ODY2IDE5OC4wMzk0NCA5OC42NDA3MDcgMjAzLjUxOTUzIDk2LjU5OTYwOSBDIDIwNy43NDYyMSA5My43MDU4NzEgMjEyLjE0ODI0IDkzLjY1Njc3OSAyMTYuNTc2MTcgOTEuODI0MjE5IEMgMjE4Ljk3OTI4IDkwLjgxNDEwOCAyMjEuNDY4MjYgODkuOTg4MjgxIDIyMi4xMDc0MiA4OS45ODgyODEgQyAyMjIuNzQ2NTcgODkuOTg4MjgxIDIyNC4xNzg5MiA4OS41MTc2ODggMjI1LjI5MTAyIDg4Ljk0MzM1OSBDIDIyNi40MDMwOCA4OC4zNjkwNTYgMjI4LjQ1NzA3IDg3LjczOTcxNiAyMjkuODUzNTIgODcuNTQyOTY5IEMgMjMxLjI0OTk5IDg3LjM0NjExMyAyMzIuMzkyNTggODcuMTEzNTI2IDIzMi4zOTI1OCA4Ny4wMjczNDQgQyAyMzIuMzkyNTggODYuOTQxMjggMjM1LjUzNzM2IDg2LjQxMDYzMSAyMzkuMzgwODYgODUuODQ3NjU2IEMgMjUwLjM4NjgxIDg0LjIzNTYxOSAyNTQuODY2MDEgODMuNTE1NTY2IDI1OS42MDkzOCA4Mi41OTk2MDkgQyAyNjIuMDM2ODcgODIuMTMwODcxIDI2Ni44MzY0NyA4MS42NjA2NTYgMjcwLjI3NTM5IDgxLjU1MjczNCBDIDI3My43MTQzNSA4MS40NDQ3MjIgMjc5LjY3MjA5IDgwLjk1MTg0OSAyODMuNTE1NjIgODAuNDU4OTg0IEMgMjg3LjM1OTE2IDc5Ljk2NjA5MSAyOTMuMzE4ODUgNzkuNDM4MDg4IDI5Ni43NTc4MSA3OS4yODUxNTYgQyAzMDAuMTk2NzcgNzkuMTMyMTYzIDMwNS4xNjEyMyA3OC42NTg5ODQgMzA3Ljc5MTAyIDc4LjIzMjQyMiBDIDMxMi42Mzk3NiA3Ny40NDU5OTMgMzI5Ljg4MzM4IDc3LjcwNzc0MiAzMzEuMzEyNSA3OC41ODk4NDQgQyAzMzEuNzQxMTggNzguODU0NDYgMzMzLjQ5MjUyIDc5LjE3MDUyIDMzNS4yMDUwOCA3OS4yOTI5NjkgQyAzNDAuNTc1MTggNzkuNjc2OTY1IDM1NC44NjkxNCA4NC4yMzA2MTYgMzU0Ljg2OTE0IDg1LjU1NjY0MSBDIDM1NC44NjkxNCA4NS44NzYzNDYgMzU1LjgxODAxIDg2LjUyODU1NSAzNTYuOTc2NTYgODcuMDA3ODEyIEMgMzYwLjAyMjcgODguMjY3ODk2IDM2NS44MzgyMSA5NC44MzM2NDIgMzY5LjAyOTMgMTAwLjYxMzI4IEMgMzcwLjU0NjUxIDEwMy4zNjExNSAzNzEuNzg5MDYgMTA2LjA0MTM2IDM3MS43ODkwNiAxMDYuNTcwMzEgQyAzNzEuNzg5MDYgMTA3LjA5OTMgMzcyLjEwNDQzIDEwNy43Mjc1NyAzNzIuNDkyMTkgMTA3Ljk2NjggQyAzNzMuMTQ3NjMgMTA4LjM3MTM0IDM3Ni4zNzU0NyAxMTQuODYwOTggMzgyLjU0ODgzIDEyOC4xODk0NSBDIDM4NS4xODQxIDEzMy44NzkwOCAzODYuMzgzMDggMTM2LjMxMjUgMzg3Ljc5Njg4IDEzOC44NDE4IEMgMzg5LjQwOTUyIDE0MS43MjY4MyAzOTQuNTkxOCAxNTMuNjg0MjYgMzk0LjU5MTggMTU0LjUxOTUzIEMgMzk0LjU5MTggMTU0Ljk4NzUyIDM5NC44OTgzNiAxNTUuMzcxMDkgMzk1LjI3MTQ4IDE1NS4zNzEwOSBDIDM5NS42NDQ2OCAxNTUuMzcxMDkgMzk2LjM0Nzk3IDE1Ni41NzgyIDM5Ni44MzU5NCAxNTguMDU0NjkgQyAzOTguMTkzMzQgMTYyLjE2MjI0IDQwMy4yNjU2MyAxNzEuNTY5MjIgNDA1Ljc5ODgzIDE3NC42Nzc3MyBDIDQwNy4wNDQxMyAxNzYuMjA1ODkgNDA4Ljc2MzY5IDE3OC44NTA1IDQwOS42MTkxNCAxODAuNTU0NjkgQyA0MTAuNDc0NTUgMTgyLjI1ODkxIDQxMi4yMzg3OCAxODUuMjgzNzcgNDEzLjU0MTAyIDE4Ny4yNzUzOSBDIDQxNC44NDMyNSAxODkuMjY3MDEgNDE2LjA5ODEyIDE5MS41ODEyOCA0MTYuMzMwMDggMTkyLjQxNzk3IEMgNDE3LjYwNTYgMTk3LjAxODYzIDQyMS40MjEyMyAyMDQuNzM3MjkgNDIzLjM4MDg2IDIwNi42ODM1OSBDIDQyNC4zMzcwNSAyMDcuNjMzMjkgNDI3LjkzNDA5IDIxMC4yMTE2MSA0MzEuMzczMDUgMjEyLjQxMjExIEMgNDM0LjgxMTk3IDIxNC42MTI1OCA0MzguMTc3NDcgMjE2Ljg2Nzg4IDQzOC44NTM1MiAyMTcuNDIzODMgQyA0MzkuNTI5NTIgMjE3Ljk3OTc4IDQ0MC42ODc3MSAyMTguNzU5OTQgNDQxLjQyNzczIDIxOS4xNTgyIEMgNDQyLjE2Nzc2IDIxOS41NTY0NyA0NDUuMDY2MzUgMjIxLjU3Njk4IDQ0Ny44NjkxNCAyMjMuNjQ2NDggQyA0NTAuNjcxODkgMjI1LjcxNjAxIDQ1Ni4zOTY4NyAyMjkuMDYwMzMgNDYwLjU4OTg0IDIzMS4wODAwOCBDIDQ2OC44NTg1NyAyMzUuMDYzMjEgNDcxLjI2NjM5IDIzNi44NjQzMyA0NzUuNjExMzMgMjQyLjMwNjY0IEMgNDc4LjczOTA5IDI0Ni4yMjQzMyA0NzkuMjc2NjQgMjQ5LjM2NzU1IDQ3Ny4yOTg4MyAyNTIuMTg3NSBDIDQ3NS43ODU5MiAyNTQuMzQ0NjQgNDcxLjI4Nzg2IDI1Ni41OTI1MiA0NjcuMzAwNzggMjU3LjE4MzU5IEMgNDY1LjU0NDIgMjU3LjQ0Mzg4IDQ2MS43ODkxIDI1OC4zMDc1NyA0NTguOTU3MDMgMjU5LjEwMTU2IEMgNDUwLjU5ODM3IDI2MS40NDUwOSA0NDYuODk5MzIgMjYyLjM1Mjc3IDQzOC40OTIxOSAyNjQuMTI4OTEgTCA0MzAuNTMzMiAyNjUuODEwNTUgTCA0MzEuMDM1MTYgMjY5LjQ2ODc1IEMgNDMxLjMxMjA1IDI3MS40ODAxMiA0MzIuNjYxMzEgMjc1LjI3MzYxIDQzNC4wMzEyNSAyNzcuODk4NDQgQyA0MzcuNjQ3NjYgMjg0LjgyNzQ0IDQzNy40NzI1MiAyODcuNzY1MzIgNDMzLjE5OTIyIDI5MS44ODY3MiBDIDQzMS4zODUyNyAyOTMuNjM2MjIgNDI5LjkwMDM5IDI5NS40MDIwNyA0MjkuOTAwMzkgMjk1LjgxMjUgQyA0MjkuOTAwMzkgMjk2LjIyMjkzIDQyOS4xOTgyOSAyOTcuNTQ0MDYgNDI4LjMzOTg0IDI5OC43NDgwNSBMIDQyNi43NzkzIDMwMC45Mzc1IEwgNDI5LjI1OTc3IDMwMy4xNDI1OCBDIDQzMC42MjM5MyAzMDQuMzU1MjYgNDMxLjE2MTEyIDMwNS4xNjc2NiA0MzAuNDUzMTIgMzA0Ljk0NzI3IEMgNDI4LjgyODg0IDMwNC40NDE2NCA0MjguNjk3OTEgMzA2LjM2ODczIDQzMC4zMTQ0NSAzMDYuOTg4MjggQyA0MzIuNDUyODQgMzA3LjgwNzc2IDQzMC43MzIwOSAzMDguNjQ3NjYgNDI3LjMyNjE3IDMwOC40NDcyNyBDIDQyMi45Mjk5NCAzMDguMTg4NDMgNDIyLjUwNjY0IDMwOS4zMDg1NyA0MjUuMTI2OTUgMzE0LjI2MTcyIEMgNDM4Ljk4OTQ3IDM0MC40NjU2MiA0NDMuMTQ4OTggMzQ4LjkyODQ0IDQ0NC4zNDE4IDM1My4zNTM1MiBDIDQ0NS43MjMzMSAzNTguNDc4NjkgNDQ1LjY0NTIxIDM2Ni4xOTQ0MSA0NDQuMTkxNDEgMzY4LjI2NzU4IEMgNDQyLjgzMjA5IDM3MC4yMDU3IDQzNC4wMzM2NCAzNzQuNzk1OTcgNDMwLjA3ODEyIDM3NS42MzA4NiBMIDQyNy4zMjYxNyAzNzYuMjEyODkgTCA0MjcuNTg3ODkgMzgxLjY3OTY5IEMgNDI3LjczMTQgMzg0LjY4NjUyIDQyOC4yOTQxMSAzODguMjIwMDkgNDI4LjgzNzg5IDM4OS41MzMyIEMgNDI5LjY3MDYgMzkxLjU0NDIxIDQyOS42NjQxOCAzOTIuMzA2OTcgNDI4LjgwMDc4IDM5NC4zNjcxOSBDIDQyOC4yMzc5MiAzOTUuNzEyNSA0MjcuMjA2MjMgMzk3LjI4MzUgNDI2LjUwNzgxIDM5Ny44NTc0MiBDIDQyNS44MDk0MyAzOTguNDMxMzEgNDIzLjUyMTgzIDQwMS4yMTQ2NSA0MjEuNDIzODMgNDA0LjA0Mjk3IEMgNDE3Ljg2MzM3IDQwOC44NDI4OCA0MTUuMTg3NSA0MTUuNzA3NDIgNDE1LjE4NzUgNDIwLjA0Mjk3IEMgNDE1LjE4NzUgNDIxLjQyODU0IDQxMy44MTQ3OCA0MjQuNjk1MTcgNDEwLjI4OTA2IDQzMS42OTkyMiBDIDQwNy4yNjI1OSA0MzcuNzExNTkgNDA3LjEyODU5IDQ0MS4zNjQ3NSA0MDkuNjY5OTIgNDQ4LjYyODkxIEMgNDEwLjY4MTM1IDQ1MS41MjAwOCA0MTEuNTA5NzcgNDU1LjI3NjE5IDQxMS41MDk3NyA0NTYuOTc0NjEgQyA0MTEuNTA5NzcgNDYwLjY1OTkyIDQwOS43NDczNyA0NjUuMTg2ODcgNDA3LjU2ODM2IDQ2Ny4wOTk2MSBDIDQwNC40MTY5MiA0NjkuODY1OTUgMzk4LjU0MzUyIDQ3My4yMjY3NiAzOTQuNTg5ODQgNDc0LjUyNzM0IEMgMzkyLjM2NDY3IDQ3NS4yNTkzMiAzODkuMzg2NzQgNDc2LjMyNTM0IDM4Ny45NzA3IDQ3Ni44OTQ1MyBDIDM4Ni4yMjk5NSA0NzcuNTk0MjcgMzgyLjg5NDEgNDc3Ljg4NTI0IDM3Ny42NzE4OCA0NzcuNzkyOTcgQyAzNjIuNDYwMzggNDc3LjUyNDAyIDM1OC44MDYwNCA0NzcuNTE1NjEgMzUxLjA4Nzg5IDQ3Ny43MzQzOCBDIDM0Ni43ODMyMiA0NzcuODU2NzYgMzQyLjMyMDM3IDQ3OC4yMDgzNCAzNDEuMTY5OTIgNDc4LjUxNTYyIEMgMzM4LjQ1ODM0IDQ3OS4yMzk4MyAzMzYuNjQ4MTcgNDgyLjQxMDI1IDMzNi4wNTA3OCA0ODcuNDg2MzMgTCAzMzUuNTc0MjIgNDkxLjUzOTA2IEwgMzQxLjg3MzA1IDQ5Ny45Mjk2OSBMIDM4OSA1NDUuNzE0ODQgQSAyODMuNDY0NTcgMjgzLjA4OTU1IDAgMCAwIDU2Ni45Mjk2OSAyODMuMDg5ODQgQSAyODMuNDY0NTcgMjgzLjA4OTU1IDAgMCAwIDI4My40NjQ4NCAwIHogIgogICAgICAgICAgIHRyYW5zZm9ybT0ibWF0cml4KDAuMjc2NjM4ODgsMCwwLDAuMjc3MDA1MzUsMzEuNzI3MjgsMzYuNzA4NjgpIiAvPgogICAgICA8L2c+CiAgICA8L2c+CiAgPC9nPgo8L3N2Zz4K"

In [ ]:
# Assign some common variables

# Text for the attribution
attribution_text = """<b>Data:</b> SMARD/Bundesnetzagentur, 
<b>Graph:</b> Jan Kühn (https://yotka.org), 
<b>License:</b> CC by 4.0"""

In [ ]:
import_files = {
    "day": "data/Realisierte_Erzeugung_201501010000_202212312359_Tag.csv",
    "month": "data/Realisierte_Erzeugung_201501010000_202212312359_Monat.csv",
    "year": "data/Realisierte_Erzeugung_201501010000_202212312359_Jahr.csv",
}

# Create empty dictionaries to be filled
data_raw = {}
data = {}

for period in import_files:

    data_raw[period] = pd.read_csv(
        import_files[period],
        sep=";",
        parse_dates=["Datum"],
        dayfirst=True,
        thousands=".",
        decimal=",",
    )

    data[period] = {}
    dfs = {}

    # Rename columns
    dfs["abs"] = data_raw[period].rename(
        columns={
            "Datum": "Date",
            "Biomasse [MWh] Berechnete Auflösungen": "Biomass",
            "Wasserkraft [MWh] Berechnete Auflösungen": "Hydro",
            "Wind Offshore [MWh] Berechnete Auflösungen": "Wind Offshore",
            "Wind Onshore [MWh] Berechnete Auflösungen": "Wind Onshore",
            "Photovoltaik [MWh] Berechnete Auflösungen": "Solar",
            "Sonstige Erneuerbare [MWh] Berechnete Auflösungen": "Other renewables",
            "Kernenergie [MWh] Berechnete Auflösungen": "Nuclear",
            "Braunkohle [MWh] Berechnete Auflösungen": "Lignite",
            "Steinkohle [MWh] Berechnete Auflösungen": "Coal",
            "Erdgas [MWh] Berechnete Auflösungen": "Gas",
            "Pumpspeicher [MWh] Berechnete Auflösungen": "Pumped storage",
            "Sonstige Konventionelle [MWh] Berechnete Auflösungen": "Other conventionals",
        }
    )

    # Drop unnecessary columns
    dfs["abs"] = dfs["abs"].drop(columns=["Anfang", "Ende"])

    # Calculate percentages
    dfs["pct"] = dfs["abs"].copy().set_index("Date")
    dfs["pct"] = dfs["pct"].div(dfs["pct"].sum(axis=1), axis=0)
    dfs["pct"] = dfs["pct"].reset_index()

    for calc in dfs:

        # Sum data for onshore and offshore wind in one column
        dfs[calc].insert(
            5, "Wind", dfs[calc]["Wind Offshore"] + dfs[calc]["Wind Onshore"]
        )

        # Sum data for coal and lignite in one column
        dfs[calc].insert(10, "Coal & Lignite", dfs[calc]["Coal"] + dfs[calc]["Lignite"])

        # Calculate totals for renewables
        dfs[calc].insert(
            15,
            "Renewables",
            dfs[calc]["Wind"]
            + dfs[calc]["Biomass"]
            + dfs[calc]["Hydro"]
            + dfs[calc]["Solar"]
            + dfs[calc]["Other renewables"],
        )

        # Calculate totals for conventionals
        dfs[calc].insert(
            16,
            "Conventionals",
            dfs[calc]["Nuclear"]
            + dfs[calc]["Coal"]
            + dfs[calc]["Gas"]
            + dfs[calc]["Pumped storage"]
            + dfs[calc]["Other conventionals"],
        )

        # Calculate totals for fossil sources
        dfs[calc].insert(
            17,
            "Fossil",
            dfs[calc]["Coal & Lignite"] + dfs[calc]["Gas"],
        )

        data[period][calc] = dfs[calc].copy()

## Yearly data
First we have a look a yearly aggregated data.

### Bar chart to show shares of production in 2022

In [ ]:
# Prepare dataframe for a bar chart
df_year_bar = data["year"]["pct"].copy()
df_year_bar = df_year_bar.drop(
    columns=[
        "Renewables",
        "Conventionals",
        "Fossil",
        "Wind Onshore",
        "Wind Offshore",
        "Lignite",
        "Coal",
    ]
)

# Substitute date column by year column
df_year_bar.insert(1, "Year", df_year_bar["Date"].dt.year)
df_year_bar = df_year_bar.drop(columns=["Date"])

df_year_bar = df_year_bar.set_index("Year").transpose()
df_year_bar = df_year_bar.sort_values(by=[2022], ascending=False)

# Have a look at the final dataframe
# df_year_bar

In [ ]:
# Plot bar chart using Plotly Express

# Set width and height
width = 900
height = 600

# Create the figure
fig = px.bar(
    df_year_bar,
    x=[2022],
    y=df_year_bar.index,
    color=df_year_bar.index,
    orientation="h",
    color_discrete_map={
        "Coal & Lignite": "#000",
        "Wind": "#109648",
        "Solar": "#109648",
        "Gas": "#000",
        "Biomass": "#109648",
        "Nuclear": "#666",
        "Hydro": "#109648",
        "Pumped storage": "#666",
        "Other conventionals": "#666",
        "Other renewables": "#109648",
    },
    text_auto=".1%",  # Show values in bars and define format
    width=width,
    height=height,
    template=custom_template,
)

# Add custom legend

# Define colors and text
colors = [
    ["#000", "Fossil"],
    ["#666", "Conventional"],
    ["#109648", "Renewable"],
]

# Define position and size of the legend
legend_top = 0.15  # Top bound (0 = bottom / 1 = top)
legend_left = 0.92  # Left bound (0 = left / 1 = right)
legend_width = 0.03
legend_height = 0.03
legend_margin = 0.3
legend_center = legend_top - legend_height / 2

# Counter for the position of legend entries
i = 0

# Create shapes (squares) and text
for color in colors:
    fig.add_shape(
        go.layout.Shape(
            type="rect",
            fillcolor=color[0],
            xref="paper",
            yref="paper",
            x0=legend_left,
            y0=legend_top - i * legend_height,
            x1=legend_left
            - (legend_width / width * height),  # Use width and height to get a square
            y1=legend_top - ((i + 1) * legend_height),
            line=dict(width=0),
        )
    )

    fig.add_annotation(
        dict(
            xref="paper",
            yref="paper",
            yanchor="middle",
            xanchor="left",
            x=legend_left,
            y=legend_center - i * legend_height,
            showarrow=False,
            text=color[1],
        )
    )
    i = i + 1 + legend_margin

# Add attribution
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="left",
    yanchor="top",
    x=-0.15,
    y=-0.07,
    showarrow=False,
    text=attribution_text,
)

# Add yotka logo
fig.add_layout_image(
    dict(
        source=logo,
        xref="paper",
        yref="paper",
        xanchor="right",
        yanchor="bottom",
        x=1.05,
        y=-0.12,
        sizex=0.1,
        sizey=0.1,
        opacity=0.8,
        layer="above",
    )
)

# Some final touches
fig.update_layout(
    title="<b>Share of electricity production in Germany 2022</b>",
    title_x=0.02,
    title_y=0.97,
    xaxis_title="",
    xaxis_tickformat=".0%",
    yaxis_title="",
    yaxis_automargin=True,
    showlegend=False,
    margin=dict(b=60, l=80, pad=5, r=30, t=60),
)

fig.show()

fig.write_image("export/1-year-2022.png", scale=2)

### Dumbbell plot showing changes from 2015 to 2022

In [ ]:
# Prepare a dataframe for the dumbbell plot
df_year_dumbbell = (
    data["year"]["pct"]
    .copy()
    .rename(
        columns={
            "Conventionals": "All conventionals",
            "Renewables": "All renewables",
            "Fossil": "All fossil sources",
        }
    )
)

# Drop unused columns
df_year_dumbbell = df_year_dumbbell.drop(
    columns=[
        "Wind Onshore",
        "Wind Offshore",
        "Lignite",
        "Coal",
        "Pumped storage",
    ]
)

# Substitute date column by year column
df_year_dumbbell.insert(1, "Year", df_year_dumbbell["Date"].dt.year)
df_year_dumbbell = df_year_dumbbell.drop(columns=["Date"])

# Keep only data for 2015 and 2022
df_year_dumbbell = df_year_dumbbell[df_year_dumbbell["Year"].isin([2015, 2022])]

# Convert dataframe to long format (melt)
df_year_dumbbell = pd.melt(
    df_year_dumbbell, id_vars=["Year"], var_name="Type", value_name="Share"
)

# Sort dataframe
df_year_dumbbell = df_year_dumbbell.sort_values(
    ["Year", "Share"], ascending=[False, True]
)

# Convert Year column to string
df_year_dumbbell["Year"] = df_year_dumbbell["Year"].astype(str)

# Have a look at the final dataframe
# df_year_dumbbell

In [ ]:
# Create Plotly figure
fig = px.scatter(
    df_year_dumbbell,
    x="Share",
    y="Type",
    color="Year",
    color_discrete_map={
        "2015": "#005f60",
        "2022": "#f78104",
    },
    category_orders={
        "Type": [
            "All conventionals",
            "All fossil sources",
            "Coal & Lignite",
            "Nuclear",
            "Gas",
            "Other conventionals",
            "",  # Create a space for the line
            "All renewables",
            "Wind",
            "Solar",
            "Biomass",
            "Hydro",
            "Other renewables",
        ],
    },
)

# Define colors by type (renewable/conventional)
colors_renewable = "#ddd"
colors_conventional = "#ddd"

# Define colors for each category/type
colors = {
    "Wind": colors_renewable,
    "Biomass": colors_renewable,
    "Hydro": colors_renewable,
    "Solar": colors_renewable,
    "Other renewables": colors_renewable,
    "Nuclear": colors_conventional,
    "Coal & Lignite": colors_conventional,
    "Gas": colors_conventional,
    "Other conventionals": colors_conventional,
    "All renewables": colors_renewable,
    "All conventionals": colors_conventional,
    "All fossil sources": colors_conventional,
}

# Iterate on type
for i in df_year_dumbbell["Type"].unique():
    # Filter by type
    df_sub = df_year_dumbbell[df_year_dumbbell["Type"] == i]

    # Assign some values
    value_2015 = df_sub["Share"].values[1]
    value_2022 = df_sub["Share"].values[0]
    category = df_sub["Type"].values[0]

    # Add a shape to connect the data points
    fig.add_shape(
        type="line",
        layer="below",
        # connect the two markers
        x0=value_2022,  # 2022 value
        x1=value_2015,  # 2015 value
        y0=category,
        y1=category,
        line=dict(
            color=colors[i],
            width=15,
        ),
    )

    # For significant changes, add value to plot

    # Calculate change
    change = (value_2022 - value_2015) * 100

    # Set threshold (in percentage points of change)
    threshold = 4

    # Calculate position next to 2022 value
    xpos = value_2022 - 0.01
    anchor = "right"

    # Change position for values below zero (negative change)
    if change < 0:
        xpos = value_2022 + 0.01
        anchor = "left"

    # Add text annotion if change is above threshold
    if abs(change) >= threshold:
        fig.add_annotation(
            xanchor=anchor,
            x=xpos,
            y=category,
            showarrow=False,
            text=f"{change:+.0f}",
        )

    # Add annotation for gas
    if category == "Gas":
        fig.add_annotation(
            xanchor="left",
            yanchor="middle",
            x=value_2022,
            y=category,
            ax=65,
            ay=10,
            showarrow=True,
            text="Gas is the only<br />fossil source that<br />increased its share",
            xshift=25,
            align="left",
            arrowhead=1,
        )

# Add horizontal line to separate conventionals and renewables
fig.add_shape(
    type="line",
    layer="below",
    x0=0,
    x1=1,
    y0="",
    y1="",
    line=dict(
        color="rgba(0, 0, 0, 0.07)",
        width=1,
    ),
)

# Add attribution
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="left",
    yanchor="top",
    x=-0.15,
    y=-0.07,
    showarrow=False,
    text=attribution_text,
)

# Add yotka logo
fig.add_layout_image(
    dict(
        source=logo,
        xref="paper",
        yref="paper",
        xanchor="right",
        yanchor="bottom",
        x=1.05,
        y=-0.12,
        sizex=0.1,
        sizey=0.1,
        opacity=0.8,
        layer="above",
    )
)

# Some final touches
fig.update_layout(
    width=900,
    height=600,
    title="<b>Change in electricity production 2015-2022 in Germany</b>",
    title_x=0.02,
    title_y=0.97,
    template=custom_template,
    margin=dict(b=60, l=80, pad=0, r=30, t=60),
    xaxis=dict(
        title="",
        tickformat=".0%",
        zeroline=False,
        gridwidth=1,
        gridcolor="rgba(0, 0, 0, 0.07)",  # Workaround, because shapes appear below grid
        range=[-0.05, 0.5],
    ),
    yaxis=dict(
        title="",
        automargin=True,
        showgrid=False,
    ),
    legend=dict(
        title="", traceorder="reversed", yanchor="bottom", y=0, xanchor="right", x=1
    ),
)

# Change marker size
fig.update_traces(
    marker=dict(
        size=15,
    ),
)

fig.show()

# Export chart as PNG image
fig.write_image("export/2-dumbbell.png", scale=2)

## Daily data: Heatmaps
The charts that got my attention in the first place were heatmaps of electricity production representing each day of the year for multiple years. I rebuild the same charts for Germany. It has to be noted, though, that there were no "coal-free" days in Germany in the observed time period. For that reason I opted to rather highlight outliers using quantiles, i.e. both 5% of the lowest and highest daily values are highlighted.

In [ ]:
# Prepare dataframe to plot the heatmaps
df_day_heatmap = data["day"]["pct"].reset_index(drop=True)

# Add columns for Year and Day of Year
df_day_heatmap.insert(1, "Year", df_day_heatmap["Date"].dt.year)
df_day_heatmap.insert(2, "Day", df_day_heatmap["Date"].dt.dayofyear)

# Have a look at the final dataframe
# df_day_heatmap

In [ ]:
# Create the heatmap charts using Plotly

# Define some settings for the different types of electricity production
metrics = [
    {
        "name": "Renewables",  # Machine readable name
        "title": "Renewables are getting closer to 50% of electricity production",  # Title
        "add": "(Wind, solar, water, biomass, and others)",  # Additional information for the subtitle
        "color": "#109648",  # Main color
        "color_low": "red",  # Color to highlight low outliers
        "color_high": "#000",  # Color to highlight high outliers
    },
    {
        "name": "Conventionals",
        "title": "Conventional electricity generation remains dominant, but declines",
        "add": "(Coal, gas, nuclear, and others)",
        "color": "#333",
        "color_low": "#109648",
        "color_high": "red",
    },
    {
        "name": "Fossil",
        "title": "No end in sight for electricity generation from fossil sources in Germany",
        "add": "(Coal, lignite, and gas)",
        "color": "#333",
        "color_low": "#109648",
        "color_high": "red",
    },
    {
        "name": "Wind",
        "title": "Wind is almost at 25% of electricity production with daily peaks up to 63%",
        "add": "",
        "color": "#D99AC5",
        "color_low": "red",
        "color_high": "#109648",
    },
    {
        "name": "Hydro",
        "title": "Hydropower does not play a major role in German electricity production",
        "add": "",
        "color": "#124E78",
        "color_low": "red",
        "color_high": "#000",
    },
    {
        "name": "Solar",
        "title": "Solar power keeps growing with lots of volatility and daily peaks up to almost 30%",
        "add": "",
        "color": "#FCF300",
        "color_low": "red",
        "color_high": "#000",
    },
    {
        "name": "Nuclear",
        "title": "Nuclear energy is on the decline",
        "add": "",
        "color": "#FF9000",
        "color_low": "#109648",
        "color_high": "#000",
    },
    {
        "name": "Coal & Lignite",
        "title": "Coal & lignite declined but were revived since 2021",
        "add": "",
        "color": "#000",
        "color_low": "#109648",
        "color_high": "red",
    },
    {
        "name": "Gas",
        "title": "Gas is on the rise with historic peaks of up to 25% in late 2022",
        "add": "",
        "color": "#D7263D",
        "color_low": "#109648",
        "color_high": "#000",
    },
]

# Start counter to be used in the loop
i = 1

# Loop through the different types
for metric in metrics:

    # Create one subplot for each year
    fig = make_subplots(
        rows=df_day_heatmap["Year"].nunique(),
        cols=1,
        vertical_spacing=0,
    )

    # Set some variables for later use
    row_num = 1  # Row number (used as a counter)
    label_pos = 0.96  # Vertical position of the first year

    # Loop through the years
    for year in df_day_heatmap["Year"].unique():

        # Filter data for current year
        data_year = df_day_heatmap[df_day_heatmap["Year"] == year]

        # Add a trace for current year
        fig.add_trace(
            go.Heatmap(
                y=data_year["Year"],
                x=data_year["Day"],
                z=data_year[metric["name"]],
                coloraxis="coloraxis",  # Use the same coloraxis for all subplots
            ),
            row=row_num,
            col=1,
        )

        # Add years as y-axis labels
        fig.add_annotation(
            xref="paper",
            yref="paper",
            xanchor="left",
            yanchor="top",
            x=-0.04,
            y=label_pos,
            showarrow=False,
            text=str(year),
            borderpad=0,
        )

        # Add yearly average value to the right
        fig.add_annotation(
            xref="paper",
            yref="paper",
            xanchor="left",
            yanchor="top",
            x=1.005,
            y=label_pos,
            showarrow=False,
            text=f"Ø {data_year[metric['name']].mean().round(2):.0%}",
            borderpad=0,
        )

        if year in [2015, 2022]:
            # Add lowest & highest days in 2015 & 2022
            df_stats = data_year[metric["name"]]

            # Define thresholds (based on all years)
            thres_low = df_day_heatmap[metric["name"]].quantile(0.05)
            thres_high = df_day_heatmap[metric["name"]].quantile(0.95)

            # Count days below/above thresholds
            count_low = df_stats[df_stats < thres_low].count()
            count_high = df_stats[df_stats > thres_high].count()

            # Set y position depending on the year
            ypos = -0.05 if year == 2015 else -0.08

            # Add annotation
            fig.add_annotation(
                xref="paper",
                yref="paper",
                xanchor="right",
                x=1,
                y=ypos,
                showarrow=False,
                text=f"{year}: <b>{count_low}</b> low days (<{thres_low:.0%}) and <b>{count_high}</b> high days (>{thres_high:.0%})",
            )

        # Increase counters
        row_num += 1
        label_pos -= 0.126

    # Add attribution
    fig.add_annotation(
        xref="paper",
        yref="paper",
        x=-0.05,
        y=-0.08,
        showarrow=False,
        text=attribution_text,
    )

    # Add yotka logo
    fig.add_layout_image(
        dict(
            source=logo,
            xref="paper",
            yref="paper",
            xanchor="right",
            yanchor="bottom",
            x=1.07,
            y=-0.08,
            sizex=0.07,
            sizey=0.07,
            opacity=0.8,
            layer="above",
        )
    )

    # Some final touches
    fig.update_layout(
        title="<b>" + metric["title"] + "</b><br />"
        "<sup>Daily electricity production share in Germany 2015-2022 "
        + metric["add"]
        + "</sup>",
        title_x=0.01,
        title_y=0.94,
        showlegend=False,
        width=1200,
        height=600,
        margin=dict(
            autoexpand=True,
            b=40,
            l=55,
            pad=0,
            r=80,
            t=100,
        ),
        template=custom_template,
        coloraxis=dict(  # Define common color axis
            cmin=0,
            cmax=df_day_heatmap[metric["name"]].max(),
            colorscale=[
                [0, metric["color_low"]],
                [  # Calculate lower bound
                    df_day_heatmap[metric["name"]].quantile(0.05)
                    / df_day_heatmap[metric["name"]].max(),
                    "white",
                ],
                [  # Calculate upper bound
                    df_day_heatmap[metric["name"]].quantile(0.95)
                    / df_day_heatmap[metric["name"]].max(),
                    metric["color"],
                ],
                [1, metric["color_high"]],
            ],
            colorbar=dict(
                xanchor="right",
                yanchor="top",
                x=1.01,
                y=1.1,
                orientation="h",
                thickness=10,
                len=0.33,
                tickformat=".0%",
            ),
        ),
    )

    # Hide ticks and labels on all axes
    fig.update_xaxes(showticklabels=False, visible=False)
    fig.update_yaxes(showticklabels=False, visible=False)

    # Export chart as PNG image
    fig.write_image(f"export/{i+2}-{metric['name']}.png", scale=2)

    fig.show()

    # Increase counter
    i += 1

## Monthly data
Interestingly, despite many efforts to boost renewable electricity production and fade out conventionals and especially fossil sources, the last two years we have observed a reversal of the trend. Due to different reasons including the Russian invasion of Ukraine and the EU's und Germany's reactions to it, fossil sources for electricity generation have gained importances again. We show that in a chart using monthly data. 

In [ ]:
# Prepare a dataframe to plot it using Plotly
df_plot_line = data["month"]["pct"].copy()

# Define width and height
width = 900
height = 600

# Create the Plotly figure
fig = px.area(
    df_plot_line,
    x="Date",
    y=["Lignite", "Coal", "Gas"],
    width=width,
    height=height,
    template=custom_template,
    color_discrete_map={
        "Coal": "#000",
        "Lignite": "#823c09",
        "Gas": "#D7263D",
    },
)

# Define dates to be highlighted
ann_dates = ["2015-10-01", "2018-02-01", "2022-12-01"]

# Loop through dates
for date in ann_dates:

    # Get the value for vertical positioning
    value = df_plot_line[df_plot_line["Date"] == date]["Fossil"]

    # Add text annotation
    fig.add_annotation(
        xanchor="right",
        yanchor="middle",
        x=date,
        y=value.values[0],
        showarrow=False,
        text=f"{date[5:7]}/{date[:4]}<br /><b>{value.values[0].round(3):.1%}</b>",
        xshift=-3,
        align="right",
    )

# Add legend inside area
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="middle",
    x=0.985,
    y=0.715,
    showarrow=False,
    text="<b>Gas</b>",
    font=dict(color="#fff"),
)

# Add legend inside area
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="middle",
    x=0.985,
    y=0.47,
    showarrow=False,
    text="<b>Coal</b>",
    font=dict(color="#fff"),
)

# Add legend inside area
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="middle",
    x=0.985,
    y=0.2,
    showarrow=False,
    text="<b>Lignite</b>",
    font=dict(color="#fff"),
)

# Add attribution
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="left",
    yanchor="top",
    x=-0.005,
    y=-0.07,
    showarrow=False,
    text=attribution_text,
)

# Add yotka logo
fig.add_layout_image(
    dict(
        source=logo,
        xref="paper",
        yref="paper",
        xanchor="right",
        yanchor="bottom",
        x=1.015,
        y=-0.1,
        sizex=0.07,
        sizey=0.07,
        opacity=0.8,
        layer="above",
    )
)

# Some final touches
fig.update_layout(
    title="<b>Fossil sources peaking again in 2022 in Germany</b><br />"
    "<sup>Share of electricity production 2015-2022 by month</sup>",
    title_x=0.02,
    title_y=0.95,
    showlegend=False,
    margin=dict(b=60, l=45, pad=5, r=20, t=70),
    xaxis=dict(title="", showgrid=False),
    yaxis=dict(title="", showgrid=False, tickformat=".0%",),
)

fig.update_traces(line=dict(width=1))

fig.show()

fig.write_image("export/12-fossil-peak-2022.png", scale=2)

In [ ]:
# Prepare a dataframe to plot it using Plotly
df_plot_line = data["month"]["pct"].copy()

# Define width and height
width = 900
height = 600

# Create the Plotly figure
fig = px.area(
    df_plot_line,
    x="Date",
    y=[
        "Nuclear",
        "Pumped storage",
        "Other conventionals",
        "Gas",
        "Lignite",
        "Coal",
        "Other renewables",
        "Biomass",
        "Hydro",
        "Solar",
        "Wind",
    ],
    width=width,
    height=height,
    template=custom_template,
    color_discrete_map={
        "Nuclear": "#FF9000",
        "Pumped storage": "#ffab2e",
        "Other conventionals": "#ffc74c",
        "Gas": "#c6c6c6",
        "Lignite": "#5e5e5e",
        "Coal": "#000",
        "Other renewables": "#94ffaf",
        "Biomass": "#77ea94",
        "Hydro": "#5acd7a",
        "Solar": "#3bb161",
        "Wind": "#109648",
    },
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.93,
    textangle=-90,
    showarrow=False,
    text="Renewable",
    font=dict(color="#666"),
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.73,
    textangle=-90,
    showarrow=False,
    text="Fossil",
    font=dict(color="#666"),
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.615,
    textangle=-90,
    showarrow=False,
    text="Other",
    font=dict(color="#666"),
)

# Add attribution
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="left",
    yanchor="top",
    x=-0.005,
    y=-0.07,
    showarrow=False,
    text=attribution_text,
)

# Add yotka logo
fig.add_layout_image(
    dict(
        source=logo,
        xref="paper",
        yref="paper",
        xanchor="left",
        yanchor="bottom",
        x=1.18,
        y=-0.1,
        sizex=0.1,
        sizey=0.1,
        opacity=0.8,
        layer="above",
    )
)

# Some final touches
fig.update_layout(
    title="<b>Still a long way to full renewable electricity production in Germany</b><br />"
    "<sup>Share of electricity production 2015-2022 by month</sup>",
    title_x=0.02,
    title_y=0.95,
    showlegend=True,
    margin=dict(b=60, l=45, pad=5, r=25, t=70),
    xaxis=dict(
        title="",
        showgrid=False,
        ticklabelposition="outside right",
        ticks="inside",
    ),
    yaxis=dict(
        title="",
        showgrid=True,
        gridcolor="rgba(0, 0, 0, 0.5)",
        tickformat=".0%",
        tickmode="array",
        tickvals=[.25, .50, .75, 1],
    ),
    legend=dict(
        title="",
        traceorder="reversed",
        yanchor="top",
        x=1.04,
        y=0.975,
    ),
)

fig.update_traces(line=dict(width=1))

fig.show()

fig.write_image("export/13-all-sources-abs.png", scale=2)

In [ ]:
# Prepare a dataframe to plot it using Plotly
df_plot_line = data["month"]["abs"].copy()

# Define width and height
width = 900
height = 600

# Create the Plotly figure
fig = px.area(
    df_plot_line,
    x="Date",
    y=[
        "Nuclear",
        "Pumped storage",
        "Other conventionals",
        "Gas",
        "Lignite",
        "Coal",
        "Other renewables",
        "Biomass",
        "Hydro",
        "Solar",
        "Wind",
    ],
    width=width,
    height=height,
    template=custom_template,
    color_discrete_map={
        "Nuclear": "#FF9000",
        "Pumped storage": "#ffab2e",
        "Other conventionals": "#ffc74c",
        "Gas": "#c6c6c6",
        "Lignite": "#5e5e5e",
        "Coal": "#000",
        "Other renewables": "#94ffaf",
        "Biomass": "#77ea94",
        "Hydro": "#5acd7a",
        "Solar": "#3bb161",
        "Wind": "#109648",
    },
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.93,
    textangle=-90,
    showarrow=False,
    text="Renewable",
    font=dict(color="#666"),
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.73,
    textangle=-90,
    showarrow=False,
    text="Fossil",
    font=dict(color="#666"),
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.615,
    textangle=-90,
    showarrow=False,
    text="Other",
    font=dict(color="#666"),
)

# Add attribution
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="left",
    yanchor="top",
    x=-0.005,
    y=-0.07,
    showarrow=False,
    text=attribution_text,
)

# Add yotka logo
fig.add_layout_image(
    dict(
        source=logo,
        xref="paper",
        yref="paper",
        xanchor="left",
        yanchor="bottom",
        x=1.18,
        y=-0.1,
        sizex=0.1,
        sizey=0.1,
        opacity=0.8,
        layer="above",
    )
)

# Some final touches
fig.update_layout(
    title="<b>Still a long way to full renewable electricity production in Germany</b><br />"
    "<sup>Electricity production 2015-2022 by month in absolute numbers</sup>",
    title_x=0.02,
    title_y=0.95,
    showlegend=True,
    margin=dict(b=60, l=45, pad=5, r=25, t=70),
    xaxis=dict(
        title="",
        showgrid=False,
        ticklabelposition="outside right",
        ticks="inside",
    ),
    yaxis=dict(
        title="",
        showgrid=True,
        gridcolor="rgba(0, 0, 0, 0.5)",
        #tickformat=".0f",
        #tickmode="array",
        #tickvals=[.25, .50, .75, 1],
    ),
    legend=dict(
        title="",
        traceorder="reversed",
        yanchor="top",
        x=1.04,
        y=0.975,
    ),
)

fig.update_traces(line=dict(width=1))

fig.show()

fig.write_image("export/14-all-sources-abs.png", scale=2)

In [ ]:
# dev = fig.full_figure_for_development()
# dev["layout"]["images"]